In [27]:
!pip install datasets evaluate seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


In [1]:
!pip install -U accelerate
!pip install -U transformers seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 795.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=cdd5c22647326ea05eb7809e652b4cc8b2a23b5dac4dd4bbce4652764f89673b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [5]:
from pathlib import Path
import re

#Function to load and split the data into text and tags
def read_data(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    #print(raw_text[:100])
    raw_docs = re.split(r'\n', raw_text)
    print(raw_docs[:100])
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            if len(line) < 3:
              continue
            token, tag, sentence= line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

#read train data
texts, tags = read_data('./data/train.tsv')

['Token\tTag\tSentence #', 'Datos\tO\tSentence 1', 'del\tO\tSentence 2', 'paciente\tO\tSentence 2', '.\tO\tSentence 2', '"', '"\tO\tSentence 2', 'Nombre\tO\tSentence 2', ':\tO\tSentence 3', ' \tO\tSentence 3', 'Ernesto\tB-NOMBRE_SUJETO_ASISTENCIA\tSentence 3', '.\tO\tSentence 3', '"', '"\tO\tSentence 3', 'Apellidos\tO\tSentence 3', ':\tO\tSentence 4', 'Rivera\tB-NOMBRE_SUJETO_ASISTENCIA\tSentence 4', 'Bueno\tI-NOMBRE_SUJETO_ASISTENCIA\tSentence 4', '.\tO\tSentence 4', '"', '"\tO\tSentence 4', 'NHC\tO\tSentence 4', ':\tO\tSentence 5', '368503\tB-ID_SUJETO_ASISTENCIA\tSentence 5', '.\tO\tSentence 5', '"', '"\tO\tSentence 5', 'NASS\tO\tSentence 5', ':\tO\tSentence 6', '26\tB-ID_ASEGURAMIENTO\tSentence 6', '63514095\tI-ID_ASEGURAMIENTO\tSentence 6', '.\tO\tSentence 6', '"', '"\tO\tSentence 6', 'Domicilio\tO\tSentence 6', ':\tO\tSentence 7', ' \tO\tSentence 7', 'Calle\tB-CALLE\tSentence 7', 'Miguel\tI-CALLE\tSentence 7', 'Benitez\tI-CALLE\tSentence 7', '90\tI-CALLE\tSentence 7', '.\tO\tSent

In [6]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
from sklearn.model_selection import train_test_split

#Split the data using an 80% random split
train_text,val_text, train_tags,  val_tags = train_test_split(texts,tags, test_size=0.2, random_state=42)

In [8]:
#Mapping the tags to label ids
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [24]:
#Get the list of unique tags in the data
label_list = list(unique_tags)
label_list

['B-ID_ASEGURAMIENTO',
 'I-HOSPITAL',
 'B-NOMBRE_SUJETO_ASISTENCIA',
 'B-TERRITORIO',
 'I-FAMILIARES_SUJETO_ASISTENCIA',
 'B-ID_SUJETO_ASISTENCIA',
 'O',
 'I-ID_ASEGURAMIENTO',
 'I-OTROS_SUJETO_ASISTENCIA',
 'B-HOSPITAL',
 'I-NUMERO_TELEFONO',
 'B-INSTITUCION',
 'B-ID_CONTACTO_ASISTENCIAL',
 'B-SEXO_SUJETO_ASISTENCIA',
 'B-CENTRO_SALUD',
 'I-SEXO_SUJETO_ASISTENCIA',
 'B-NOMBRE_PERSONAL_SANITARIO',
 'I-ID_TITULACION_PERSONAL_SANITARIO',
 'B-CALLE',
 'B-PROFESION',
 'B-NUMERO_TELEFONO',
 'I-ID_SUJETO_ASISTENCIA',
 'B-FECHAS',
 'B-NUMERO_FAX',
 'I-CORREO_ELECTRONICO',
 'I-PROFESION',
 'I-NOMBRE_PERSONAL_SANITARIO',
 'I-EDAD_SUJETO_ASISTENCIA',
 'B-FAMILIARES_SUJETO_ASISTENCIA',
 'I-CALLE',
 'B-PAIS',
 'I-TERRITORIO',
 'Tag',
 'B-OTROS_SUJETO_ASISTENCIA',
 'I-CENTRO_SALUD',
 'I-NOMBRE_SUJETO_ASISTENCIA',
 'B-CORREO_ELECTRONICO',
 'B-ID_TITULACION_PERSONAL_SANITARIO',
 'I-INSTITUCION',
 'B-EDAD_SUJETO_ASISTENCIA',
 'I-PAIS',
 'I-FECHAS',
 'I-NUMERO_FAX']

In [10]:
from transformers import BertTokenizerFast

#Loading the BERT Tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')

#Obtaining the training and validation encodings after tokenization of text
train_encodings = tokenizer(train_text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [11]:
import numpy as np

def encode_tags(tags, encodings):
  print(tags[:10])
  labels = [[tag2id[tag] for tag in doc] for doc in tags]
  encoded_labels = []
  for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
    doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
    arr_offset = np.array(doc_offset)

    # set labels whose first offset position is 0 and the second is not 0
    doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
    encoded_labels.append(doc_enc_labels.tolist())

  return encoded_labels

#Obtaining the training and validation labels after encoding of tags
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

[['O'], ['I-CALLE'], ['O'], ['I-CALLE'], ['O'], ['O'], ['B-PAIS'], ['O'], ['O'], ['O']]
[['O'], ['O'], [], ['O'], ['O'], [], ['O'], ['B-ID_SUJETO_ASISTENCIA'], ['O'], ['O']]


In [12]:
import torch

class MEDOCCANDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = MEDOCCANDataset(train_encodings, train_labels)
val_dataset = MEDOCCANDataset(val_encodings, val_labels)

In [13]:
from transformers import BertForTokenClassification

#Loading the modelfrom transformers library and sending to cuda
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(unique_tags))
model = model.to(device)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import evaluate

seqeval = evaluate.load("seqeval")

#compute_metrics funciton is used to obtain the various metrics during training and evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [29]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.model_selection import KFold
from transformers import Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup

# Defining test dataset
test_texts, test_tags = read_data('test.tsv')
test_encodings = tokenizer(test_texts, is_split_into_words=True,
                          return_offsets_mapping=True, padding=True,
                           truncation=True)
test_labels = encode_tags(test_tags, test_encodings)
test_dataset = MEDOCCANDataset(test_encodings, test_labels)  # Replace with your actual test dataset

# Instantiate Trainer and TrainingArguments for this fold
training_args = TrainingArguments(
    output_dir=f'./results',  # Directory for results
    num_train_epochs=5,                  # Total number of training epochs
    per_device_train_batch_size=12,       # Batch size per GPU
    logging_dir=f'./logs',    # Directory for storing logs
    save_strategy = "epoch",                       # Save model checkpoint every 500 steps
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    logging_steps=100,                    # Log metrics every 100 steps
    learning_rate=3e-5,                   # Learning rate
    gradient_accumulation_steps=1,        # Number of updates steps before backward pass
    weight_decay=0.0,                     # Weight decay (if applicable)
    adam_beta1=0.9,                       # AdamW beta1
    adam_beta2=0.999,                     # AdamW beta2
    adam_epsilon=1e-8,                    # AdamW epsilon
    max_grad_norm=1.0,                    # Gradient clipping threshold
    warmup_steps=500,                     # Number of warmup steps for the scheduler
    load_best_model_at_end=True,          # Load the best model when training ends
    metric_for_best_model='eval_loss',    # Metric to use to determine the best model
    greater_is_better=False               # Indicate if higher metric values are better
)

# Instantiate AdamW optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate,
                        betas=(training_args.adam_beta1, training_args.adam_beta2),
                        eps=training_args.adam_epsilon)
num_training_steps = len(train_encodings) // (training_args.per_device_train_batch_size *
                                                    training_args.gradient_accumulation_steps) * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=training_args.warmup_steps,
                                                num_training_steps=num_training_steps)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    optimizers=(optimizer,scheduler),
    train_dataset= train_dataset,
    eval_dataset= val_dataset,
    compute_metrics=compute_metrics
)

# Train the model for this fold
trainer.train()

# Evaluate the model for this fold
evaluation_result = trainer.evaluate()
print(f"Evaluation result:")
print(evaluation_result)
test_predictions = trainer.predict(test_dataset)
model.save_pretrained(f"./model/")


['Token\tTag\tSentence #', 'Datos\tO\tSentence 1', 'del\tO\tSentence 2', 'paciente\tO\tSentence 2', '.\tO\tSentence 2', '"', '"\tO\tSentence 2', 'Nombre\tO\tSentence 2', ':\tO\tSentence 3', ' \tO\tSentence 3', 'Ignacio\tB-NOMBRE_SUJETO_ASISTENCIA\tSentence 3', '.\tO\tSentence 3', '"', '"\tO\tSentence 3', 'Apellidos\tO\tSentence 3', ':\tO\tSentence 4', 'Rico\tB-NOMBRE_SUJETO_ASISTENCIA\tSentence 4', 'Pedroza\tI-NOMBRE_SUJETO_ASISTENCIA\tSentence 4', '.\tO\tSentence 4', '"', '"\tO\tSentence 4', 'NHC\tO\tSentence 4', ':\tO\tSentence 5', '5467980\tB-ID_SUJETO_ASISTENCIA\tSentence 5', '.\tO\tSentence 5', '"', '"\tO\tSentence 5', 'Domicilio\tO\tSentence 5', ':\tO\tSentence 6', 'Av.\tB-CALLE\tSentence 6', 'Beniarda\tI-CALLE\tSentence 6', ',\tI-CALLE\tSentence 6', '13\tI-CALLE\tSentence 6', '.\tO\tSentence 6', '"', '"\tO\tSentence 6', 'Localidad/\tO\tSentence 6', 'Provincia\tO\tSentence 7', ':\tO\tSentence 7', 'Valencia\tB-TERRITORIO\tSentence 7', '.\tO\tSentence 7', '"', '"\tO\tSentence 7', '

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.483900,0.581712,0.739350,0.231151,0.352193,0.931418
2,0.493600,0.581712,0.739350,0.231151,0.352193,0.931418
3,0.531700,0.581712,0.739350,0.231151,0.352193,0.931418
4,0.520100,0.581712,0.739350,0.231151,0.352193,0.931418
5,0.703200,0.581712,0.739350,0.231151,0.352193,0.931418


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation result:
{'eval_loss': 0.5817118287086487, 'eval_precision': 0.7393501805054151, 'eval_recall': 0.2311512415349887, 'eval_f1': 0.3521926053310404, 'eval_accuracy': 0.9314180858994209, 'eval_runtime': 113.6956, 'eval_samples_per_second': 479.104, 'eval_steps_per_second': 59.888, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Tag seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [45]:
!git lfs install --system --skip-repo

Git LFS initialized.


In [50]:
tokenizer.save_pretrained('./tokenizer')

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

In [52]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [54]:
model.push_to_hub('sravn/NER-BERT-MEDOCCAN')

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sravn/NER-BERT-MEDOCCAN/commit/dd4298273b29e5400ee3898f420df9f029d025f9', commit_message='Upload BertForTokenClassification', commit_description='', oid='dd4298273b29e5400ee3898f420df9f029d025f9', pr_url=None, pr_revision=None, pr_num=None)

In [55]:
tokenizer.push_to_hub('sravn/NER-BERT-MEDOCCAN')

CommitInfo(commit_url='https://huggingface.co/sravn/NER-BERT-MEDOCCAN/commit/718a27825b44ec4ef1b607db2b6831bf5d78f773', commit_message='Upload tokenizer', commit_description='', oid='718a27825b44ec4ef1b607db2b6831bf5d78f773', pr_url=None, pr_revision=None, pr_num=None)

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
model.save_pretrained("/content/drive/My Drive/NER_BERT/model/")
tokenizer.save_pretrained("/content/drive/My Drive/NER_BERT/tokenizer/")

('/content/drive/My Drive/NER_BERT/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/NER_BERT/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/NER_BERT/tokenizer/vocab.txt',
 '/content/drive/My Drive/NER_BERT/tokenizer/added_tokens.json',
 '/content/drive/My Drive/NER_BERT/tokenizer/tokenizer.json')